# Scraping data about the best games according to metacritic 

### Libraries

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import seaborn as sns

### Scraping code

In [4]:
s = Service('C:\webdrivers\chromedriver.exe')
driver = webdriver.Chrome(service=s)
url = 'https://www.metacritic.com/browse/games/score/metascore/all/all/filtered'
driver.get(url)
driver.implicitly_wait(10)
cookies = driver.find_element(By.XPATH,'//button[@id="onetrust-accept-btn-handler"]')
cookies.click()
driver.implicitly_wait(10)
all_games_links = []
games_data = []
games_links = driver.find_elements(By.XPATH,'//a[@class="title"]')
for game_link in games_links:
    game_link = game_link.get_attribute('href')
    all_games_links.append({'link':game_link})
for k in all_games_links:
    print(f'''Getting data about {k['link'].split('/')[5]}. Waiting...''')
    driver.get(k["link"])
    driver.implicitly_wait(5)
    game_title = driver.find_element(By.XPATH,'//div[@class="product_title"]//a').text.strip()
    try:
        game_summary_expand_button = driver.find_element(By.XPATH,'(//ul[@class="summary_details"])[2]//span[@class="toggle_expand_collapse toggle_expand"]').click()
        game_summary = driver.find_element(By.XPATH,'(//ul[@class="summary_details"])[2]//span[@class="blurb blurb_expanded"]').text.strip()
    except:
        game_summary = driver.find_element(By.XPATH,'(//ul[@class="summary_details"])[2]//span[@class="data"]').text.strip()
    metacritic_score = driver.find_element(By.XPATH,'//span[@itemprop="ratingValue"]').text.strip()
    users_score = driver.find_element(By.XPATH,'(//a[@class="metascore_anchor"])[2]//div').text.strip()
    num_of_user_reviews = driver.find_element(By.XPATH,'(//span[@class="count"])[2]').text.strip()
    game_dev = driver.find_element(By.XPATH,'(//ul[@class="summary_details"])[3]//li[1]').text.strip()
    game_genres = driver.find_element(By.XPATH,'(//ul[@class="summary_details"])[3]//li[2]').text.strip()
    try:
        buy_game = driver.find_element(By.XPATH,'//a[@rel="nofollow noopener"]').get_attribute('href')
    except:
        buy_game = 'No available offer'
    games_data.append({'title':game_title,
                       'summary':game_summary,
                       'metacritic_score':metacritic_score,
                       'users_score':users_score,
                       'num_of_user_reviews':num_of_user_reviews,
                       'dev':game_dev,
                       'genres':game_genres,
                       'buy_game':buy_game})
    time.sleep(2)
    driver.refresh()


Getting data about the-legend-of-zelda-ocarina-of-time. Waiting...
Getting data about tony-hawks-pro-skater-2. Waiting...
Getting data about grand-theft-auto-iv. Waiting...
Getting data about soulcalibur. Waiting...
Getting data about grand-theft-auto-iv. Waiting...
Getting data about super-mario-galaxy. Waiting...
Getting data about super-mario-galaxy-2. Waiting...
Getting data about red-dead-redemption-2. Waiting...
Getting data about grand-theft-auto-v. Waiting...
Getting data about grand-theft-auto-v. Waiting...
Getting data about disco-elysium-the-final-cut. Waiting...
Getting data about grand-theft-auto-v. Waiting...
Getting data about tony-hawks-pro-skater-2. Waiting...
Getting data about the-legend-of-zelda-breath-of-the-wild. Waiting...
Getting data about tony-hawks-pro-skater-3. Waiting...
Getting data about perfect-dark. Waiting...
Getting data about red-dead-redemption-2. Waiting...
Getting data about grand-theft-auto-v. Waiting...
Getting data about metroid-prime. Waiting.

In [5]:
games_data

[{'title': 'THE LEGEND OF ZELDA: OCARINA OF TIME',
  'summary': 'As a young boy, Link is tricked by Ganondorf, the King of the Gerudo Thieves. The evil human uses Link to gain access to the Sacred Realm, where he places his tainted hands on Triforce and transforms the beautiful Hyrulean landscape into a barren wasteland. Link is determined to fix the problems he helped to create, so with the help of Rauru he travels through time gathering the powers of the Seven Sages.',
  'metacritic_score': '99',
  'users_score': '9.1',
  'num_of_user_reviews': 'based on 8119 Ratings',
  'dev': 'Developer: Nintendo',
  'genres': 'Genre(s): Action Adventure, Fantasy',
  'buy_game': 'https://www.amazon.com/dp/B003ZU0PHU%3Ftag%3Dmetacritic-gamescat-20%26linkCode%3Dosi%26th%3D1%26psc%3D1'},
 {'title': "TONY HAWK'S PRO SKATER 2",
  'summary': "As most major publishers' development efforts shift to any number of next-generation platforms, Tony Hawk 2 will likely stand as one of the last truly fantastic gam

In [6]:
df = pd.DataFrame(games_data)

In [8]:
df

,title,summary,metacritic_score,users_score,num_of_user_reviews,dev,genres,buy_game
0,THE LEGEND OF ZELDA: OCARINA OF TIME,"As a young boy, Link is tricked by Ganondorf, ...",99,9.1,based on 8119 Ratings,Developer: Nintendo,"Genre(s): Action Adventure, Fantasy",https://www.amazon.com/dp/B003ZU0PHU%3Ftag%3Dm...
1,TONY HAWK'S PRO SKATER 2,As most major publishers' development efforts ...,98,7.5,based on 924 Ratings,Developer: Neversoft Entertainment,"Genre(s): Sports, Alternative, Skateboarding",https://www.amazon.com/dp/B00004TEXU%3Ftag%3Dm...
2,GRAND THEFT AUTO IV,[Metacritic's 2008 PS3 Game of the Year; Also ...,98,7.9,based on 5046 Ratings,Developer: Rockstar North,"Genre(s): Action Adventure, Modern, Modern, Op...",https://www.amazon.com/dp/B000HKP88C%3Ftag%3Dm...
3,SOULCALIBUR,"This is a tale of souls and swords, transcendi...",98,8.5,based on 425 Ratings,Developer: Namco,"Genre(s): Action, Fighting, 3D",https://www.amazon.com/dp/B00000K125%3Ftag%3Dm...
4,GRAND THEFT AUTO IV,[Metacritic's 2008 Xbox 360 Game of the Year; ...,98,8.0,based on 3754 Ratings,Developer: Rockstar North,"Genre(s): Action Adventure, Modern, Modern, Op...",https://www.amazon.com/dp/B000FRU1UM%3Ftag%3Dm...
...,...,...,...,...,...,...,...,...
95,PERSONA 5 ROYAL,"Forced to transfer to a high school in Tokyo, ...",94,8.4,based on 297 Ratings,Developer: Atlus,"Genre(s): Role-Playing, Japanese-Style",https://www.amazon.com/dp/B0BFDZDTJ7%3Ftag%3Dm...
96,SID MEIER'S CIVILIZATION II,An empire-building turn-based strategy game. T...,94,8.7,based on 509 Ratings,Developer: MPS Labs,"Genre(s): Strategy, Turn-Based, Historic, Gene...",https://www.amazon.com/dp/B00002S8AV%3Ftag%3Dm...
97,THE WITCHER 3: WILD HUNT - COMPLETE EDITION,"You are Geralt of Rivia, mercenary monster sla...",94,8.0,based on 601 Ratings,Developer: CD Projekt Red Studio,"Genre(s): Role-Playing, Action RPG",https://www.amazon.com/dp/B01K6010FM%3Ftag%3Dm...
98,QUAKE,Rage through 32 single player levels and 6 dea...,94,8.8,based on 1025 Ratings,Developer: id Software,"Genre(s): Action, Shooter, Shooter, First-Pers...",No available offer


In [10]:
ranking = []
for i in range(1,101):
    ranking.append({'rank':i})

In [11]:
df_rank = pd.DataFrame(ranking)

In [12]:
df = pd.concat([df_rank,df],axis=1)

### Final data frame

In [13]:
df

,rank,title,summary,metacritic_score,users_score,num_of_user_reviews,dev,genres,buy_game
0,1,THE LEGEND OF ZELDA: OCARINA OF TIME,"As a young boy, Link is tricked by Ganondorf, ...",99,9.1,based on 8119 Ratings,Developer: Nintendo,"Genre(s): Action Adventure, Fantasy",https://www.amazon.com/dp/B003ZU0PHU%3Ftag%3Dm...
1,2,TONY HAWK'S PRO SKATER 2,As most major publishers' development efforts ...,98,7.5,based on 924 Ratings,Developer: Neversoft Entertainment,"Genre(s): Sports, Alternative, Skateboarding",https://www.amazon.com/dp/B00004TEXU%3Ftag%3Dm...
2,3,GRAND THEFT AUTO IV,[Metacritic's 2008 PS3 Game of the Year; Also ...,98,7.9,based on 5046 Ratings,Developer: Rockstar North,"Genre(s): Action Adventure, Modern, Modern, Op...",https://www.amazon.com/dp/B000HKP88C%3Ftag%3Dm...
3,4,SOULCALIBUR,"This is a tale of souls and swords, transcendi...",98,8.5,based on 425 Ratings,Developer: Namco,"Genre(s): Action, Fighting, 3D",https://www.amazon.com/dp/B00000K125%3Ftag%3Dm...
4,5,GRAND THEFT AUTO IV,[Metacritic's 2008 Xbox 360 Game of the Year; ...,98,8.0,based on 3754 Ratings,Developer: Rockstar North,"Genre(s): Action Adventure, Modern, Modern, Op...",https://www.amazon.com/dp/B000FRU1UM%3Ftag%3Dm...
...,...,...,...,...,...,...,...,...,...
95,96,PERSONA 5 ROYAL,"Forced to transfer to a high school in Tokyo, ...",94,8.4,based on 297 Ratings,Developer: Atlus,"Genre(s): Role-Playing, Japanese-Style",https://www.amazon.com/dp/B0BFDZDTJ7%3Ftag%3Dm...
96,97,SID MEIER'S CIVILIZATION II,An empire-building turn-based strategy game. T...,94,8.7,based on 509 Ratings,Developer: MPS Labs,"Genre(s): Strategy, Turn-Based, Historic, Gene...",https://www.amazon.com/dp/B00002S8AV%3Ftag%3Dm...
97,98,THE WITCHER 3: WILD HUNT - COMPLETE EDITION,"You are Geralt of Rivia, mercenary monster sla...",94,8.0,based on 601 Ratings,Developer: CD Projekt Red Studio,"Genre(s): Role-Playing, Action RPG",https://www.amazon.com/dp/B01K6010FM%3Ftag%3Dm...
98,99,QUAKE,Rage through 32 single player levels and 6 dea...,94,8.8,based on 1025 Ratings,Developer: id Software,"Genre(s): Action, Shooter, Shooter, First-Pers...",No available offer
